In [ ]:
import nflreadpy as nfl
import pandas as pd
from sqlalchemy import create_engine


windows = [3, 5, 10, 15] 

years = list(range(2016, 2026))

numeric_exclude = ['season', 'week']
exclude = numeric_exclude + ['team', 'opponent_team']

#Basic player stats
player_stats = nfl.load_player_stats(years).to_pandas()
player_keep = ['player_id', 'player_display_name', 'season', 'week', 'position', 'team', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'fantasy_points']
player_stats = player_stats[player_keep]
player_stats = player_stats[player_stats['position'] == 'QB']
player_stats = player_stats.sort_values(['player_id', 'season', 'week'])
player_stats['fumbles_lost'] = player_stats['sack_fumbles_lost'] + player_stats['rushing_fumbles_lost']
player_stats.drop(columns=['sack_fumbles_lost', 'rushing_fumbles_lost'], inplace=True)


#Next gen player stats
next_gen = nfl.load_nextgen_stats(years).to_pandas()
next_gen_keep = ['season', 'week', 'avg_time_to_throw', 'avg_completed_air_yards', 'avg_intended_air_yards', 'avg_air_yards_differential', 'aggressiveness', 'max_completed_air_distance', 'avg_air_yards_to_sticks', 'passer_rating', 'completion_percentage', 'expected_completion_percentage', 'avg_air_distance', 'max_air_distance', 'player_gsis_id']
next_gen = next_gen[next_gen_keep]
next_gen = next_gen.sort_values(['player_gsis_id', 'season', 'week'])



#Schedule dataframe
schedule = nfl.load_schedules(years).to_pandas()
schedule_keep = ['game_id', 'season', 'week', 'away_team', 'home_team', 'total_line', 'div_game', 'temp', 'wind', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name']
schedule = schedule[schedule_keep]


#team stats (to be used for player)
team_stats = nfl.load_team_stats(years).to_pandas()
team_stats_keep = ['season', 'week', 'team', 'opponent_team', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'receiving_epa', 'def_tackles_with_assist', 'def_tackles_for_loss', 'def_fumbles_forced', 'def_sacks', 'def_qb_hits', 'def_interceptions', 'def_pass_defended', 'def_tds', 'misc_yards', 'penalty_yards', 'fg_made', 'pat_att']
team_stats = team_stats[team_stats_keep]
team_stats = team_stats.sort_values(['team', 'season', 'week'])

team_stats.rename(columns={col: f'team_{col}' for col in team_stats.columns if col not in exclude}, inplace=True)



In [ ]:
#build spine

home_players = schedule[['game_id', 'season', 'week', 'home_team', 'away_team', 'home_qb_id', 'home_qb_name', 'total_line', 'temp', 'wind']]
home_players.columns = ['game_id', 'season', 'week', 'team', 'opponent', 'player_id', 'player_name', 'line', 'temp', 'wind']


away_players = schedule[['game_id', 'season', 'week', 'away_team', 'home_team', 'away_qb_id', 'away_qb_name', 'total_line', 'temp', 'wind']]
away_players.columns = ['game_id', 'season', 'week', 'team', 'opponent', 'player_id', 'player_name', 'line', 'temp', 'wind']


model_spine = pd.concat([home_players, away_players], ignore_index=True)

In [ ]:

model_spine = model_spine.merge(player_stats.rename(columns={'opponent_team': 'opponent'}), how='left', on=['season', 'week', 'team', 'opponent', 'player_id'])
model_spine = model_spine.merge(next_gen.rename(columns={'player_gsis_id': 'player_id'}), how='left', on=['season', 'week', 'player_id'])
model_spine = model_spine.merge(team_stats, how='left', on=['season', 'week', 'team'])
model_spine = model_spine.sort_values(by=['player_id', 'season', 'week'])


In [ ]:

targets = ['passing_yards', 'passing_tds', 'passing_interceptions', 'fumbles_lost', 'rushing_yards', 'rushing_tds']


numeric_exclude = ['season', 'week', 'wind', 'temp', 'line']
numeric = model_spine.select_dtypes(include='number').drop(columns=numeric_exclude)

#rolling averages
for window in windows:
    rolling_df = model_spine.groupby('player_id')[numeric.columns].transform(lambda x: x.rolling(window=window, min_periods=1).mean()).add_suffix(f'_last_{window}').shift(1)
    model_spine = pd.concat([model_spine, rolling_df], axis=1)


model_spine = model_spine.drop(
    model_spine[(model_spine['season'] < 2025) & (model_spine[targets].isnull().any(axis=1))].index 
)

model_spine.drop(columns=['player_display_name', 'opponent_team'], inplace=True)
model_spine.rename(columns={'player_name': 'player_display_name'}, inplace=True)
df = model_spine.copy()

print("Final dataframe shape:", df.shape)
print("Columns in final dataframe:", df.columns.tolist())
print(df.head())

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
conn_string = os.getenv('DATABASE_URL')
engine = create_engine(conn_string)

df.to_sql('qb_features_raw', engine, if_exists='replace', index=False)

In [ ]:
import pandas as pd
import xgboost as xgb
import mlflow
from sklearn.metrics import mean_absolute_error
from sqlalchemy import text
import numpy as np
from scipy.stats import poisson


# quantile predictions
target_configs = {
    'quantile': {
        'targets': ['passing_yards', 'rushing_yards'],
        'params': {
            'objective': 'reg:quantileerror',
            'quantile_alpha': [0.25, 0.6, 0.75],
            'tree_method': 'hist',
            'multi_strategy': 'one_output_per_tree',
            'min_child_weight': 0,
            'max_depth': 6,
            'reg_alpha': 0, 
            'reg_lambda': 1
        }
    },
    'poisson': {
        'targets': ['passing_tds', 'passing_interceptions', 'fumbles_lost', 'rushing_tds'],
        'params': {
            'objective': 'count:poisson',
            'tree_method': 'hist',
            'max_depth': 8,
            'learning_rate': 0.03,
            'reg_alpha': 0.1,
            'reg_lambda': 0.5,
            'max_delta_step': 1, 
            'min_child_weight': 1
        }
    }
}


features = [col for col in df.columns if 'last_' in col]


featured_players = ['00-0034869', '00-0036355', '00-0036389', '00-0039918', '00-0039851', '00-0034857', '00-0039732', '00-0033106', '00-0034855', '00-0026498']


# Filter for training data
train_base = df[(df['season'] >= 2016) & (df['season'] <= 2024)].copy()

# Dictionary to store trained models and selected features for each target
trained_models = {}
target_feature_map = {}

print("Training historical models (2016-2024)...")
for target in targets:
    target_train = train_base.dropna(subset=[target])
    train_mean = target_train[target].mean()
    
    #first pass for feature pruning
    selector = xgb.XGBRegressor(objective='reg:quantileerror', quantile_alpha=0.5)
    selector.fit(target_train[features], target_train[target])
    
    #top 30 for training, top 5 for importance tracking
    scores = selector.get_booster().get_score(importance_type='gain')
    top_30_feats = [f[0] for f in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:30]]
    top_5_feats = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]
    
    #second pass for final model training
    current_cfg = next((cfg for cfg in target_configs.values() if target in cfg['targets']), None)
    m = xgb.XGBRegressor(**current_cfg['params'], base_score=train_mean)
    m.fit(target_train[top_30_feats], target_train[target])
    
    #store results
    trained_models[target] = m
    target_feature_map[target] = {'top_30': top_30_feats, 'top_5': top_5_feats}




In [ ]:
#trackers
all_projections = []
all_importances = [] 
cumulative_tracker_actual = {t: {} for t in targets}
cumulative_tracker_pred = {t: {} for t in targets}
games_played_tracker = {} 

featured_players = ['00-0034869', '00-0036355', '00-0036389', '00-0039918', '00-0039851', '00-0034857', '00-0039732', '00-0033106', '00-0033077', '00-0026498']


weeks_2025 = range(1, 18) 

for current_week in sorted(weeks_2025):
    test_df = df[(df['season'] == 2025) & (df['week'] == current_week)].copy()
    if test_df.empty: continue

    week_proj = test_df[test_df['player_id'].isin(featured_players)].copy()

    with mlflow.start_run(run_name=f'QB_Inference_2025_W{current_week}'):
        for target in targets:
            #predictions
            m = trained_models[target]
            selected_cols = target_feature_map[target]['top_30']
            top_features_5 = target_feature_map[target]['top_5']
            raw_preds = m.predict(week_proj[selected_cols])

            #low, med, high interval logic
            if 'count:poisson' in str(m.get_params()['objective']):
                week_proj[f'{target}_pred_med'] = raw_preds
                week_proj[f'{target}_pred_low'] = poisson.ppf(0.25, raw_preds)
                week_proj[f'{target}_pred_high'] = poisson.ppf(0.75, raw_preds)
            else:
                sorted_preds = np.sort(raw_preds, axis=1)
                week_proj[f'{target}_pred_low'] = sorted_preds[:, 0]
                week_proj[f'{target}_pred_med'] = sorted_preds[:, 1]
                week_proj[f'{target}_pred_high'] = sorted_preds[:, 2]

            #check week to see if have actual data to add to cumulative totals
            is_future_week = week_proj[target].isnull().all()

            if not is_future_week:
                week_proj[f'{target}_actual'] = week_proj[target]
                target_actuals, target_model_preds, target_baselines = [], [], []

                for idx, row in week_proj.iterrows():
                    p_id = row['player_id']
                    actual = row[target]
                    pred_med = row[f'{target}_pred_med']
                    
                    games_played = games_played_tracker.get(p_id, 0)
                    player_avg = (cumulative_tracker_actual[target].get(p_id, 0)) / games_played if games_played > 0 else 0
                    
                    if target == targets[0]: 
                        games_played_tracker[p_id] = games_played + 1
                    
                    cumulative_tracker_actual[target][p_id] = cumulative_tracker_actual[target].get(p_id, 0) + actual
                    cumulative_tracker_pred[target][p_id] = cumulative_tracker_pred[target].get(p_id, 0) + pred_med
                    
                    week_proj.at[idx, f'{target}_cum_actual'] = cumulative_tracker_actual[target][p_id]
                    week_proj.at[idx, f'{target}_cum_pred'] = cumulative_tracker_pred[target][p_id]
                    week_proj.at[idx, 'games_played_to_date'] = games_played_tracker[p_id]
                    
                    target_actuals.append(actual)
                    target_model_preds.append(pred_med)
                    target_baselines.append(player_avg)
                #evaluate model
                #mae_m = mean_absolute_error(target_actuals, target_model_preds)
                #mae_b = mean_absolute_error(target_actuals, target_baselines)
    

                #feature importances
                for feat_name, gain_val in top_features_5:
                    for _, p_row in week_proj.iterrows():
                        all_importances.append({
                            'week': current_week, 'player_id': p_row['player_id'],
                            'target': target, 'feature_name': feat_name,
                            'importance_gain': gain_val, 'player_feature_value': p_row[feat_name]
                        })
            else:
                #fill future weeks with NaNs so the column selection doesn't break
                week_proj[f'{target}_actual'] = np.nan
                week_proj[f'{target}_cum_actual'] = np.nan
                week_proj[f'{target}_cum_pred'] = np.nan
                week_proj['games_played_to_date'] = games_played_tracker.get(row['player_id'], 0) if 'row' in locals() else 0

        target_cols = []
        for t in targets:
            target_cols += [f'{t}_pred_low', f'{t}_pred_med', f'{t}_pred_high', 
                           f'{t}_actual', f'{t}_cum_actual', f'{t}_cum_pred']
        
        essential_cols = ['player_id', 'player_display_name', 'opponent', 'season', 'week']
        all_projections.append(week_proj[essential_cols + target_cols])

# Save to SQL
if all_projections:
    final_projections_df = pd.concat(all_projections, ignore_index=True)
    with engine.begin() as conn:
        final_projections_df.to_sql('weekly_projections2', conn, if_exists='replace', index=False)

In [ ]:
import joblib

joblib.dump(trained_models, 'qb_trained_models.joblib')
joblib.dump(target_feature_map, 'qb_target_feature_map.joblib')